In [6]:
#!/usr/bin/env python
# coding: utf-8

import arcpy

# -------------------------------------------------------------------
# User-defined parameters
# -------------------------------------------------------------------
parcels_fc = r"C:\Users\Sea Grant\Documents\GitHub\WW_Overlay_2024\Data\Outputs\TaxPlat_No_P1\TaxPlat_No_P1.shp"
sewer_coverage_fc = r"C:\Users\Sea Grant\Documents\GitHub\WW_Overlay_2024\Data\HonoluluSewerBuffer_v2\HonoluluSewerBuffer_v2.shp"
parcel_id_field = "plat"  # Unique identifier field in your Parcels
coverage_threshold = 50  # We want parcels with <= 50% coverage

# Temporary in-memory datasets
temp_intersect_fc = "in_memory\\temp_intersect"
temp_stats_table = "in_memory\\temp_stats"

# -------------------------------------------------------------------
# 1. Intersect the Parcels with Sewer Coverage
# -------------------------------------------------------------------
arcpy.env.overwriteOutput = True

print("Performing intersection...")
arcpy.analysis.Intersect(
    in_features=[parcels_fc, sewer_coverage_fc],
    out_feature_class=temp_intersect_fc,
    join_attributes="ALL",
    cluster_tolerance="",
    output_type="INPUT"
)

# Check the number of intersected features
intersect_count = int(arcpy.management.GetCount(temp_intersect_fc)[0])
print(f"Number of intersected features: {intersect_count}")

if intersect_count == 0:
    print("No intersected features found. Please check if the parcels and sewer coverage overlap.")
    exit()

# -------------------------------------------------------------------
# 2. Add and Calculate Intersection Area
# -------------------------------------------------------------------
intersect_area_field = "IntersectArea"
arcpy.management.AddField(temp_intersect_fc, intersect_area_field, "DOUBLE")

print("Calculating intersected areas...")
arcpy.management.CalculateGeometryAttributes(
    in_features=temp_intersect_fc,
    geometry_property=[[intersect_area_field, "AREA_GEODESIC"]],
    area_unit="SQUARE_METERS"
)

# -------------------------------------------------------------------
# 3. Ensure Parcels Have a ParcelArea Field
# -------------------------------------------------------------------
parcel_area_field = "ParcelArea"
if parcel_area_field not in [f.name for f in arcpy.ListFields(parcels_fc)]:
    print("Adding parcel area field...")
    arcpy.management.AddField(parcels_fc, parcel_area_field, "DOUBLE")
    arcpy.management.CalculateGeometryAttributes(
        input_features=parcels_fc,
        geometry_property=[[parcel_area_field, "AREA_GEODESIC"]],
        area_unit="SQUARE_METERS"
    )

# -------------------------------------------------------------------
# 4. Summarize Intersection by Parcel ID
# -------------------------------------------------------------------
print("Summarizing intersected areas...")
arcpy.analysis.Statistics(
    in_table=temp_intersect_fc,
    out_table=temp_stats_table,
    statistics_fields=[[intersect_area_field, "SUM"]],
    case_field=parcel_id_field
)

# Check if statistics table has entries
stats_count = int(arcpy.management.GetCount(temp_stats_table)[0])
print(f"Number of entries in stats table: {stats_count}")

# -------------------------------------------------------------------
# 5. Build a Dictionary of {ParcelID: Sum_IntersectArea}
# -------------------------------------------------------------------
sum_dict = {}
fields_in_stats = [f.name for f in arcpy.ListFields(temp_stats_table)]
print(f"Fields in stats table: {fields_in_stats}")

with arcpy.da.SearchCursor(temp_stats_table, [parcel_id_field, f"SUM_{intersect_area_field}"]) as cur:
    for row in cur:
        sum_dict[row[0]] = row[1]

if not sum_dict:
    print("The sum_dict is empty. This indicates that no intersect areas were summarized.")
    exit()

# -------------------------------------------------------------------
# 6. Update Each Parcel with the Coverage Percentage
# -------------------------------------------------------------------
coverage_ratio_field = "CovPct"
if coverage_ratio_field not in [f.name for f in arcpy.ListFields(parcels_fc)]:
    arcpy.management.AddField(parcels_fc, coverage_ratio_field, "DOUBLE")

print("Updating parcel coverage percentages...")
with arcpy.da.UpdateCursor(parcels_fc, [parcel_id_field, parcel_area_field, coverage_ratio_field]) as ucur:
    for parcel_row in ucur:
        p_id = parcel_row[0]  # Parcel ID
        p_area = parcel_row[1]  # Parcel Area

        # Get the total intersected area (default 0 if not in dictionary)
        intersect_area_sum = sum_dict.get(p_id, 0.0)

        # Compute coverage ratio (as percent)
        coverage_ratio = (intersect_area_sum / p_area) * 100 if p_area > 0 else 0.0

        # Update the attribute table
        parcel_row[2] = coverage_ratio
        ucur.updateRow(parcel_row)

print("Coverage percentages have been written to the attribute table.")


Performing intersection...
Number of intersected features: 220528
Calculating intersected areas...
Summarizing intersected areas...
Number of entries in stats table: 171
Fields in stats table: ['FID', 'plat', 'FREQUENCY', 'SUM_IntersectArea']
Updating parcel coverage percentages...
Coverage percentages have been written to the attribute table.
